In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

# 1.x Save Video

In [370]:
cap = cv2.VideoCapture(0)

#비디오 속성 확인
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS) #초당 프레임수

#비디오 레코드 저장
videoWriter= cv2.VideoWriter('train2.avi',cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()
    
    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
        
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF ==ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

# 2. Capture Landmarks & Export to CSV

In [371]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [372]:
landmarks=['shol']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [373]:
# 데이터 저장할 csv 파일 형성
#with open('coords2.csv',mode='w',newline='') as f:
    #csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    #csv_writer.writerow(landmarks)

In [374]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        
        if action=='good':
            keypoints = np.insert(keypoints,0,0)

        if action=='bad':
            keypoints = np.insert(keypoints,0,1)
            
        with open('coords2.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        pass

In [ ]:
# 영상을 통해 데이터셋 구축
cap =cv2.VideoCapture("train2.avi") #setting video capture device(number은 웹캠을 대표하는 숫자)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret,frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))
        
        #키보드로 입력한 키와 동일한 아스키코드값
        k = cv2.waitKey(1)
        
        if k == 103: # good의 g
            print('good')
            export_landmark(results,'good')
            
        if k == 98: # bad의 b
            print('bad')
            export_landmark(results,'bad')
            

        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

good
bad
bad
good
good
bad
bad
good
good
good
bad
bad
bad
bad
bad
good
bad
good
bad
bad
good
good
bad
bad
good
bad
bad
good
bad
bad
good
bad
bad
good
bad
bad
good
bad
bad
good
bad
bad
good
bad


# 3. Train Custom Model Using Scikit Learn

In [361]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [362]:
df = pd.read_csv('coords2.csv')

In [363]:
df[df['shol']==1] #어깨 치우침

,shol,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1,1.0,0.487129,0.549459,-0.489963,0.999937,0.505937,0.495354,-0.463815,0.999828,0.519188,...,0.339190,0.000007,0.614137,2.815218,-0.001046,0.000047,0.480860,2.805694,-0.039083,0.000028
4,1.0,0.567956,0.559312,-0.459783,0.999975,0.587326,0.506150,-0.428856,0.999935,0.599676,...,0.235501,0.000005,0.557422,2.826785,0.055146,0.000165,0.410211,2.806959,-0.158027,0.000032
7,1.0,0.457475,0.557121,-0.534636,0.999941,0.478374,0.496359,-0.509733,0.999823,0.493544,...,0.411791,0.000005,0.607968,2.856014,-0.044150,0.000036,0.471196,2.851840,0.002930,0.000023
10,1.0,0.581302,0.531183,-0.508463,0.999973,0.602649,0.478343,-0.474910,0.999932,0.615067,...,0.236471,0.000007,0.570711,2.887246,0.097844,0.000148,0.418127,2.858220,-0.159401,0.000032
13,1.0,0.543590,0.516945,-0.527500,0.999985,0.566193,0.461015,-0.497758,0.999961,0.578725,...,0.234134,0.000002,0.573421,2.889021,-0.019457,0.000032,0.430530,2.855451,-0.168483,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,1.0,0.590321,0.630558,-0.501212,0.999952,0.615777,0.563498,-0.463990,0.999855,0.628213,...,0.149621,0.000004,0.606567,3.220519,-0.170773,0.000041,0.466942,3.188442,-0.295844,0.000019
1946,1.0,0.594387,0.630672,-0.526178,0.999977,0.618741,0.570172,-0.488984,0.999925,0.631848,...,0.210978,0.000002,0.608009,3.217775,-0.084493,0.000048,0.459894,3.185964,-0.224985,0.000015
1947,1.0,0.525677,0.628462,-0.854992,0.999805,0.551166,0.556182,-0.809964,0.999645,0.567732,...,-0.014464,0.000070,0.713005,3.204692,-0.483462,0.000179,0.569759,3.198621,-0.512950,0.000164
1949,1.0,0.568020,0.623097,-0.532951,0.999967,0.585238,0.555765,-0.490286,0.999915,0.596595,...,0.356966,0.000003,0.660567,3.187879,-0.057214,0.000027,0.509372,3.175371,-0.106761,0.000012


In [364]:
X = df.drop('shol',axis=1) #shol을 제외한 입력 feature만 남김.
y = df['shol']

In [365]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123) #어깨

In [366]:
y_test

1638    0.0
1241    0.0
958     0.0
1738    0.0
1697    1.0
       ... 
1912    1.0
840     1.0
1908    1.0
1293    0.0
850     0.0
Name: shol, Length: 586, dtype: float64

https://camo.githubusercontent.com/37bc1d0f546f2dd001798a0f66057c9c907e6b5efae95a068bcce8c852c2e234/68747470733a2f2f692e696d6775722e636f6d2f336a38425064632e706e67

In [367]:
df.drop(['x13'],axis=1,inplace=True)
df.drop(['y13'],axis=1,inplace=True)
df.drop(['z13'],axis=1,inplace=True)
df.drop(['v13'],axis=1,inplace=True)

df.drop(['x14'],axis=1,inplace=True)
df.drop(['y14'],axis=1,inplace=True)
df.drop(['z14'],axis=1,inplace=True)
df.drop(['v14'],axis=1,inplace=True)

df.drop(['x15'],axis=1,inplace=True)
df.drop(['y15'],axis=1,inplace=True)
df.drop(['z15'],axis=1,inplace=True)
df.drop(['v15'],axis=1,inplace=True)

df.drop(['x16'],axis=1,inplace=True)
df.drop(['y16'],axis=1,inplace=True)
df.drop(['z16'],axis=1,inplace=True)
df.drop(['v16'],axis=1,inplace=True)

df.drop(['x17'],axis=1,inplace=True)
df.drop(['y17'],axis=1,inplace=True)
df.drop(['z17'],axis=1,inplace=True)
df.drop(['v17'],axis=1,inplace=True)

df.drop(['x18'],axis=1,inplace=True)
df.drop(['y18'],axis=1,inplace=True)
df.drop(['z18'],axis=1,inplace=True)
df.drop(['v18'],axis=1,inplace=True)

df.drop(['x19'],axis=1,inplace=True)
df.drop(['y19'],axis=1,inplace=True)
df.drop(['z19'],axis=1,inplace=True)
df.drop(['v19'],axis=1,inplace=True)

df.drop(['x20'],axis=1,inplace=True)
df.drop(['y20'],axis=1,inplace=True)
df.drop(['z20'],axis=1,inplace=True)
df.drop(['v20'],axis=1,inplace=True)

df.drop(['x21'],axis=1,inplace=True)
df.drop(['y21'],axis=1,inplace=True)
df.drop(['z21'],axis=1,inplace=True)
df.drop(['v21'],axis=1,inplace=True)

df.drop(['x22'],axis=1,inplace=True)
df.drop(['y22'],axis=1,inplace=True)
df.drop(['z22'],axis=1,inplace=True)
df.drop(['v22'],axis=1,inplace=True)

df.drop(['x23'],axis=1,inplace=True)
df.drop(['y23'],axis=1,inplace=True)
df.drop(['z23'],axis=1,inplace=True)
df.drop(['v23'],axis=1,inplace=True)

df.drop(['x24'],axis=1,inplace=True)
df.drop(['y24'],axis=1,inplace=True)
df.drop(['z24'],axis=1,inplace=True)
df.drop(['v24'],axis=1,inplace=True)

df.drop(['x25'],axis=1,inplace=True)
df.drop(['y25'],axis=1,inplace=True)
df.drop(['z25'],axis=1,inplace=True)
df.drop(['v25'],axis=1,inplace=True)

df.drop(['x26'],axis=1,inplace=True)
df.drop(['y26'],axis=1,inplace=True)
df.drop(['z26'],axis=1,inplace=True)
df.drop(['v26'],axis=1,inplace=True)

df.drop(['x27'],axis=1,inplace=True)
df.drop(['y27'],axis=1,inplace=True)
df.drop(['z27'],axis=1,inplace=True)
df.drop(['v27'],axis=1,inplace=True)

df.drop(['x28'],axis=1,inplace=True)
df.drop(['y28'],axis=1,inplace=True)
df.drop(['z28'],axis=1,inplace=True)
df.drop(['v28'],axis=1,inplace=True)

df.drop(['x29'],axis=1,inplace=True)
df.drop(['y29'],axis=1,inplace=True)
df.drop(['z29'],axis=1,inplace=True)
df.drop(['v29'],axis=1,inplace=True)

df.drop(['x30'],axis=1,inplace=True)
df.drop(['y30'],axis=1,inplace=True)
df.drop(['z30'],axis=1,inplace=True)
df.drop(['v30'],axis=1,inplace=True)

df.drop(['x31'],axis=1,inplace=True)
df.drop(['y31'],axis=1,inplace=True)
df.drop(['z31'],axis=1,inplace=True)
df.drop(['v31'],axis=1,inplace=True)

df.drop(['x32'],axis=1,inplace=True)
df.drop(['y32'],axis=1,inplace=True)
df.drop(['z32'],axis=1,inplace=True)
df.drop(['v32'],axis=1,inplace=True)

df.drop(['x33'],axis=1,inplace=True)
df.drop(['y33'],axis=1,inplace=True)
df.drop(['z33'],axis=1,inplace=True)
df.drop(['v33'],axis=1,inplace=True)

In [368]:
df

,shol,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z10,v10,x11,y11,z11,v11,x12,y12,z12,v12
0,0.0,0.529603,0.561145,-0.514029,0.999973,0.548111,0.508904,-0.477052,0.999931,0.559385,...,-0.431849,0.999947,0.499266,0.624254,-0.445067,0.999968,0.663548,0.882354,-0.147571,0.998671
1,1.0,0.487129,0.549459,-0.489963,0.999937,0.505937,0.495354,-0.463815,0.999828,0.519188,...,-0.413122,0.999894,0.463420,0.612799,-0.409795,0.999892,0.655467,0.810649,-0.178291,0.998994
2,0.0,0.513576,0.558442,-0.528455,0.999966,0.533987,0.499218,-0.493996,0.999909,0.545779,...,-0.449971,0.999934,0.486582,0.626315,-0.453213,0.999946,0.678620,0.901397,-0.173842,0.998732
3,0.0,0.528299,0.551363,-0.531231,0.999979,0.549499,0.495418,-0.495506,0.999945,0.561634,...,-0.449803,0.999962,0.497912,0.617606,-0.454625,0.999975,0.678400,0.901221,-0.168827,0.998792
4,1.0,0.567956,0.559312,-0.459783,0.999975,0.587326,0.506150,-0.428856,0.999935,0.599676,...,-0.388406,0.999947,0.543459,0.620897,-0.402808,0.999976,0.677966,0.883018,-0.100888,0.999197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,0.0,0.550860,0.616721,-0.581138,0.999958,0.578499,0.554660,-0.539683,0.999894,0.593350,...,-0.491971,0.999932,0.520794,0.688855,-0.483649,0.999922,0.749248,0.972166,-0.192070,0.998386
1949,1.0,0.568020,0.623097,-0.532951,0.999967,0.585238,0.555765,-0.490286,0.999915,0.596595,...,-0.449135,0.999919,0.534981,0.701705,-0.457113,0.999936,0.714352,0.967207,-0.145602,0.997663
1950,0.0,0.560104,0.618246,-0.544445,0.999973,0.582610,0.550929,-0.505959,0.999927,0.595376,...,-0.460811,0.999931,0.529822,0.693943,-0.450518,0.999943,0.750869,0.978997,-0.177124,0.997846
1951,1.0,0.555805,0.622163,-0.816786,0.999679,0.583188,0.551796,-0.783666,0.999221,0.597880,...,-0.714018,0.999498,0.532528,0.695114,-0.686157,0.999629,0.778946,0.968769,-0.343463,0.982579


In [369]:
df.mean()

shol    0.489503
x1      0.553421
y1      0.566453
z1     -0.662718
v1      0.999850
x2      0.574767
y2      0.505418
z2     -0.625198
v2      0.999652
x3      0.588368
y3      0.506817
z3     -0.625117
v3      0.999716
x4      0.601149
y4      0.509019
z4     -0.625182
v4      0.999607
x5      0.533644
y5      0.504826
z5     -0.627276
v5      0.999696
x6      0.519811
y6      0.505526
z6     -0.626724
v6      0.999766
x7      0.507480
y7      0.506754
z7     -0.626969
v7      0.999719
x8      0.621148
y8      0.543767
z8     -0.363362
v8      0.999618
x9      0.492849
y9      0.540090
z9     -0.359229
v9      0.999801
x10     0.579396
y10     0.635738
z10    -0.562505
v10     0.999791
x11     0.528294
y11     0.633569
z11    -0.561655
v11     0.999848
x12     0.729598
y12     0.892248
z12    -0.191567
v12     0.995822
dtype: float64

In [340]:
for val in range(1,12+1):
    print(df['v{}'.format(val)].mean())

0.9998552939458022
0.9996596268181356
0.9997219654399326
0.999617078669021
0.9997028067018779
0.9997720056946108
0.9997269191768844
0.9996245802279241
0.9998074447071601
0.9998034302660352
0.9998601358308297
0.9963487984677696


## 3.2 Train Machine Learning Classification Model

In [341]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [342]:
pipelines = {
    #'lr' : make_pipeline(StandardScaler(),LogisticRegression(max_iter=1300)),
    #'rc' : make_pipeline(StandardScaler(),RidgeClassifier(max_iter=1300)),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier(max_features=2000)),
    #'gb' : make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=1300))
}

In [343]:
fit_models={}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [344]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier',
                  RandomForestClassifier(max_features=2000))])}

In [345]:
fit_models['rf'].predict(X_test)

array([1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 0.

## 3.3 Evaluate and Serialize Model

In [346]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [347]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,pos_label=1),
         recall_score(y_test.values,yhat,pos_label=1))

rf 0.9138840070298769 0.9140625 0.896551724137931


In [348]:
yhat = fit_models['rf'].predict(X_test)

In [349]:
yhat[:10]

array([1., 0., 0., 0., 0., 0., 0., 1., 1., 1.])

In [350]:
y_test

1576    1.0
235     0.0
655     0.0
1353    0.0
1215    1.0
       ... 
1835    1.0
1483    0.0
1320    1.0
141     1.0
408     0.0
Name: shol, Length: 569, dtype: float64

In [351]:
with open('Shoulder.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 4. Make Detections with Model

In [352]:
with open('Shoulder.pkl','rb') as f:
    model = pickle.load(f)

In [353]:
landmarks=['shol']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [354]:
#VISUALIZE DEGREE
cap =cv2.VideoCapture(0) #setting video capture device(number은 웹캠을 대표하는 숫자)

# Curl counter variables
counter = 0

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret, frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환
        
         #Extract landmarks
        try:  
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 0 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Good'
            elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Shoulder'
                counter +=1
            #if body_language_class == 0 and body_language_prob[body_language_prob.argmin()] <= .3:
                #current_stage = 'Good'
            #elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmin()] <= .3:
                #current_stage = 'Shoulder'
                #counter +=1
        
            #Setup status bow
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1) 


            #지금 상태
            cv2.putText(image,'CLASS',(15,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image, 'Shoulder' if body_language_class == 1 else 'Good',(15,40)
                        ,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Stage data 
            cv2.putText(image,'Count',(180,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    
            cv2.putText(image,str(counter),(175,40),
                       cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Render detections
            #drawing 툴을 사용해서 감지된 자세포인트와 이들 간의 포인트를 연결해 보여준다.
            #mp_drawing.DrawingSpec은 관절부위와 bone 부분의 색깔, 굵기 등을 지정해준다.
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                     mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))


        except: #error가 있으면 실행x
            pass
        
        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.